In [ ]:

%run includes/2-compute-setup.ipynb

# Compute Instances
Summary and detail reports on compute instances and persistent disks deployed across the GCP estate.

In [ ]:

# get project/regions and usage summary info for each project
queries = [
    f"""
    SELECT name as 'region',
    '{project}' as project,
    zones,
    quotas
    FROM google.compute.regions WHERE project = '{project}'
    """
    for project in all_projects
]

regions_df = run_stackql_queries(queries)
regions_df, compute_projects = get_compute_projects(regions_df)

# get all instances across all projects
queries = [
    f"""
    SELECT 
    name,
    status,
    SPLIT_PART(machineType, '/', 11) as machineType,
    '{project}' as project,
    creationTimestamp,
    lastStartTimestamp,
    SPLIT_PART(JSON_EXTRACT(networkInterfaces, '$[0].network'), '/', 10) as network,
    SPLIT_PART(JSON_EXTRACT(networkInterfaces, '$[0].subnetwork'), '/', 11) as subnetwork,
    JSON_EXTRACT(networkInterfaces, '$[0].networkIP') as networkIP,
    JSON_EXTRACT(tags, '$.items') as tags,
    labels
    FROM google.compute.instances
    WHERE project = '{project}'
    """
    for project in compute_projects
]
instances_df = run_stackql_queries(queries)

## Compute Summary

In [ ]:

show_compute_summary_cards(regions_df, len(instances_df))

## Compute Usage Summary by Project

In [ ]:

plot_compute_metrics(regions_df, ['INSTANCES'], 400)

In [ ]:

plot_compute_metrics(regions_df, ['CPUS', 'C2_CPUS', 'DISKS_TOTAL_GB'], 250)

In [ ]:

# plot_compute_metrics(regions_df, ['INSTANCE_GROUPS', 'INSTANCE_GROUP_MANAGERS', 'REGIONAL_INSTANCE_GROUP_MANAGERS'], 250)

## Instances by Machine Type

In [ ]:

plot_machine_types(instances_df)

In [ ]:

# get all persistent disks across all projects
queries = [
    f"""
    SELECT 
    name, sizeGb
    FROM google.compute.disks
    WHERE project = '{project}'
    """
    for project in compute_projects
]
persistent_disks_df = run_stackql_queries(queries)	

## Persistent Disk Sizes

In [ ]:

plot_persistent_disks(persistent_disks_df)

## Instances Detail

In [ ]:

instances_df

## Persistent Disks Detail

In [ ]:

persistent_disks_df

## GKE Clusters

In [ ]:
queries = [
    f"""
    SELECT *, '{project}' as project
    FROM google.container.clusters
    WHERE projectsId = '{project}' AND locationsId = 'australia-southeast1'
    """
    for project in all_projects
]
clusters_df = run_stackql_queries(queries)
clusters_df	